In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np

import keras
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.xception import preprocess_input

from xception import get_xception

Using TensorFlow backend.


In [3]:
data_dir = '/home/ubuntu/data/'

In [4]:
data_generator = ImageDataGenerator(
    data_format='channels_last',
    preprocessing_function=preprocess_input
)

train_generator = data_generator.flow_from_directory(
    data_dir + 'train', 
    target_size=(299, 299),
    batch_size=64
)

val_generator = data_generator.flow_from_directory(
    data_dir + 'val', 
    target_size=(299, 299),
    batch_size=64
)

Found 25600 images belonging to 256 classes.
Found 5120 images belonging to 256 classes.


# Model

In [5]:
model = get_xception()

for layer in model.layers[:-7]:
    layer.trainable = False

# Training

In [6]:
model.compile(
    optimizer=optimizers.Adam(lr=1e-4), 
    loss='categorical_crossentropy', metrics=['accuracy', 'top_k_categorical_accuracy']
)

In [7]:
model.fit_generator(
    train_generator, 
    steps_per_epoch=400, epochs=10, verbose=1,
    validation_data=val_generator, validation_steps=48
)

Epoch 1/10
400/400 [==============================] - 205s - loss: 3.7803 - acc: 0.4745 - top_k_categorical_accuracy: 0.6484 - val_loss: 1.9980 - val_acc: 0.7826 - val_top_k_categorical_accuracy: 0.9297
Epoch 2/10
400/400 [==============================] - 204s - loss: 1.5028 - acc: 0.7947 - top_k_categorical_accuracy: 0.9386 - val_loss: 1.0635 - val_acc: 0.8206 - val_top_k_categorical_accuracy: 0.9473
Epoch 3/10
400/400 [==============================] - 204s - loss: 0.8996 - acc: 0.8484 - top_k_categorical_accuracy: 0.9615 - val_loss: 0.8329 - val_acc: 0.8320 - val_top_k_categorical_accuracy: 0.9535
Epoch 4/10
400/400 [==============================] - 204s - loss: 0.6544 - acc: 0.8775 - top_k_categorical_accuracy: 0.9739 - val_loss: 0.7266 - val_acc: 0.8402 - val_top_k_categorical_accuracy: 0.9528
Epoch 5/10
400/400 [==============================] - 204s - loss: 0.5098 - acc: 0.9024 - top_k_categorical_accuracy: 0.9819 - val_loss: 0.6648 - val_acc: 0.8389 - val_top_k_categorical_ac

# Results

In [8]:
val_generator_no_shuffle = data_generator.flow_from_directory(
    data_dir + 'val', 
    target_size=(299, 299),
    batch_size=64, shuffle=False
)

Found 5120 images belonging to 256 classes.


In [9]:
model.evaluate_generator(val_generator_no_shuffle, 80)

[0.57773708868771789, 0.85429687499999996, 0.95703125]

In [10]:
model.save_weights('xception_weights.hdf5')